# AI Assistant Agent for Booking Appointments on Google Calendar for a Dental Clinic


# 1. Set Up Google Calendar API Integration
Steps:

1-Create a Google Cloud Project and Enable API:

*Go to the Google Cloud Console.
*Create a new project or use an existing one.
*Navigate to the API & Services dashboard.
*Click + ENABLE APIS AND SERVICES.
*Search for "Google Calendar API" and enable it.

2-Set Up OAuth 2.0 Credentials:

*Go to Credentials in the API & Services section.
*Click Create Credentials and select OAuth Client ID.
*Configure the consent screen if prompted.
*Choose Desktop App as the application type.
*Download the credentials.json file.

3- Write Python Script for API Authentication and Interaction:
   Install the required libraries:

In [5]:
pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib


In [7]:
# Use the following Python script to authenticate and interact with the Google Calendar API:

In [ ]:
from __future__ import print_function
import datetime
import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.discovery import build
import pickle

SCOPES = ['https://www.googleapis.com/auth/calendar']

def authenticate_google_calendar():
    creds = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            creds = pickle.load(token)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            creds = flow.run_local_server(port=0)
        with open('token.pickle', 'wb') as token:
            pickle.dump(creds, token)
    service = build('calendar', 'v3', credentials=creds)
    return service

def create_event(service, summary, start_time, end_time):
    event = {
        'summary': summary,
        'start': {
            'dateTime': start_time,
            'timeZone': 'America/New_York',
        },
        'end': {
            'dateTime': end_time,
            'timeZone': 'America/New_York',
        },
    }
    event = service.events().insert(calendarId='primary', body=event).execute()
    print(f"Event created: {event.get('htmlLink')}")

if __name__ == '__main__':
    service = authenticate_google_calendar()
    create_event(service, 'Dental Appointment', '2024-08-10T10:00:00', '2024-08-10T11:00:00')


Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=994216741718-347abfbojmfiie0ektdmde8jp6ej7uvq.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A65362%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcalendar&state=yN4XX4wrrEiyFOJz0wIX0iouj63ufS&access_type=offline


# Deliverables:
Python script for Google Calendar API authentication and appointment booking.

# 2. Build a Conversational Agent Using LangChain
Steps:

1-Set Up LangChain:

Install LangChain:

In [ ]:
pip install langchain

2-Define the Conversation Flow:

Use LangChain to create a conversational agent. Below is a simplified example to handle basic intents:

In [ ]:
from langchain.chains import ConversationChain
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.memory import SimpleMemory

# Define the prompt
prompt_template = PromptTemplate(
    input_variables=["user_input"],
    template="You are a dental clinic assistant. Your job is to help users book, reschedule, or cancel appointments. {user_input}"
)

# Initialize the model
model = OpenAI(model="text-davinci-003")

# Initialize the conversation chain
chain = ConversationChain(
    llm=model,
    prompt=prompt_template,
    memory=SimpleMemory()
)

# Example function to handle a user query
def handle_user_input(user_input):
    response = chain.run(user_input=user_input)
    return response


# Deliverables:

LangChain-based conversational agent script.

# 3. Implement Conversation Flow and Data Handling

**Steps:**

1. Define the conversation flow for booking, rescheduling, and canceling appointments.
2. Implement data handling to store and manage appointment details.
3. Ensure the agent can handle different scenarios like unavailability of time slots and patient preferen
   ces.
Steps:

1-Define the Conversation Flow:
  Implement a function to handle the conversation and integrate it with the Google Calendar API functions. For example:

# 1. Define the Conversation Flow for Booking, Rescheduling, and Canceling Appointments
Here's a simple conversation flow for each scenario:

Booking Appointment

Ask for the patient's name and preferred date.
Ask for the preferred time.
Check availability and confirm the appointment.

Rescheduling Appointment

Ask for the existing appointment details.
Ask for the new preferred date and time.
Check availability and confirm the new appointment.

Canceling Appointment

Ask for the appointment details to be canceled.
Confirm the cancellation.

In [ ]:
from datetime import datetime

def handle_booking_request(user_input):
    # Extract details from user_input (e.g., date, time)
    # For simplicity, assuming fixed date and time
    appointment_date = '2024-08-10'
    appointment_time = '10:00:00'
    start_time = f"{appointment_date}T{appointment_time}"
    end_time = f"{appointment_date}T{appointment_time[:-2]}1:00:00"
    
    service = authenticate_google_calendar()
    create_event(service, 'Dental Appointment', start_time, end_time)
    return "Your appointment has been booked successfully!"

# Example conversation handling
user_input = "I would like to book an appointment on August 10th at 10 AM."
response = handle_booking_request(user_input)
print(response)


 2- Handle Different Scenarios:
* Unavailability of Time Slots: Check available slots before booking.
* Patient Preferences: Adapt to different patient preferences (e.g., time, date).

# 2. Implement Data Handling to Store and Manage Appointment Details
We can use Python's built-in data structures to store appointment details. For simplicity, we'll use a dictionary to manage appointments.

# Deliverables:
# Python Script Implementation
Here's a Python script that handles booking, rescheduling, and canceling appointments:
Python script with implemented conversation flow and data handling.

In [ ]:
from datetime import datetime, timedelta

# Sample appointment data structure
appointments = {}

def is_date_valid(date_str):
    try:
        datetime.strptime(date_str, "%Y-%m-%d")
        return True
    except ValueError:
        return False

def is_time_valid(time_str):
    try:
        datetime.strptime(time_str, "%H:%M")
        return True
    except ValueError:
        return False

def check_availability(date_str, time_str):
    appointment_datetime = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
    return appointment_datetime not in appointments

def book_appointment(name, date_str, time_str):
    if not is_date_valid(date_str):
        return "Invalid date format. Please use YYYY-MM-DD."
    if not is_time_valid(time_str):
        return "Invalid time format. Please use HH:MM."
    if check_availability(date_str, time_str):
        appointment_datetime = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
        appointments[appointment_datetime] = name
        return f"Appointment booked for {name} on {date_str} at {time_str}."
    else:
        return "Sorry, this time slot is not available."

def reschedule_appointment(name, old_date_str, old_time_str, new_date_str, new_time_str):
    old_appointment_datetime = datetime.strptime(f"{old_date_str} {old_time_str}", "%Y-%m-%d %H:%M")
    if old_appointment_datetime in appointments and appointments[old_appointment_datetime] == name:
        if check_availability(new_date_str, new_time_str):
            del appointments[old_appointment_datetime]
            new_appointment_datetime = datetime.strptime(f"{new_date_str} {new_time_str}", "%Y-%m-%d %H:%M")
            appointments[new_appointment_datetime] = name
            return f"Appointment rescheduled for {name} to {new_date_str} at {new_time_str}."
        else:
            return "Sorry, this new time slot is not available."
    else:
        return "No appointment found to reschedule."

def cancel_appointment(name, date_str, time_str):
    appointment_datetime = datetime.strptime(f"{date_str} {time_str}", "%Y-%m-%d %H:%M")
    if appointment_datetime in appointments and appointments[appointment_datetime] == name:
        del appointments[appointment_datetime]
        return f"Appointment canceled for {name} on {date_str} at {time_str}."
    else:
        return "No appointment found to cancel."

def handle_request(request_type, *args):
    if request_type == "book":
        return book_appointment(*args)
    elif request_type == "reschedule":
        return reschedule_appointment(*args)
    elif request_type == "cancel":
        return cancel_appointment(*args)
    else:
        return "Invalid request type."

# Example usage
print(handle_request("book", "John Doe", "2024-08-10", "10:00"))
print(handle_request("reschedule", "John Doe", "2024-08-10", "10:00", "2024-08-10", "11:00"))
print(handle_request("cancel", "John Doe", "2024-08-10", "11:00"))


# 4. Test the Agent with Sample Conversations
Steps:

1-Test Various Scenarios:
 *Test successful bookings, rescheduling, and cancellations.
 *Check handling of unavailable slots and user preferences.
2-Document Issues and Improvements:
  *Track any issues faced during testing and document improvements made.

# We can test the above script by running various scenarios: 

In [ ]:
# Booking a New Appointment:
print(handle_request("book", "Alice Smith", "2024-08-10", "10:00"))

In [ ]:
# Rescheduling an Existing Appointment:
print(handle_request("reschedule", "Alice Smith", "2024-08-10", "10:00", "2024-08-11", "09:00"))

In [ ]:
# Canceling an Appointment:
print(handle_request("cancel", "Alice Smith", "2024-08-11", "09:00"))

# Deliverables
Python Script: The provided script handles booking, rescheduling, and canceling appointments with basic validation and availability checking.
Test Results and Documentation: You should document any issues encountered during testing, such as invalid input handling or unavailability of time slots. Note any necessary improvements based on test results.